In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import tensorflow as tf

from baseline import Baseline
from system_t import System_T

# Load Fashion-MNIST dataset from keras

In [2]:
fashion = tf.keras.datasets.fashion_mnist
(fashion_train_images,fashion_train_labels), (fashion_test_images, fashion_test_labels) = fashion.load_data()

fashion_train_labels = fashion_train_labels
fashion_test_labels = fashion_test_labels
fashion_train_images = fashion_train_images.reshape((60000, 28, 28, 1)) / 255.0
fashion_test_images = fashion_test_images.reshape((10000, 28, 28, 1)) / 255.0

train_images = fashion_train_images
train_labels = fashion_train_labels

# Basic setting: slices have the same amounts of data

In [3]:
def shuffle(data, label):
    shuffle = np.arange(len(data))
    np.random.shuffle(shuffle)
    data = data[shuffle]
    label = label[shuffle]
    return data, label

num_class = len(np.unique(train_labels))
print("Number of slices : %d" % num_class)

y_train_one_hot = tf.keras.utils.to_categorical(train_labels)
mixed_data = (train_images, y_train_one_hot)

initial_data_array = []
val_data_dict = []
add_data_dict = []
    
val_data_num = 500

for i in range(num_class):
    data_num = 400
    initial_data_array.append(data_num)
    idx = np.argmax(mixed_data[1], axis=1) == i

    val_data_dict.append((mixed_data[0][idx][data_num:data_num+val_data_num], mixed_data[1][idx][data_num:data_num+val_data_num]))
    add_data_dict.append((mixed_data[0][idx][data_num+val_data_num:], mixed_data[1][idx][data_num+val_data_num:]))

    if i == 0:
        train_data = mixed_data[0][idx][:data_num]
        train_label = mixed_data[1][idx][:data_num]
        val_data = mixed_data[0][idx][data_num:data_num+val_data_num]
        val_label = mixed_data[1][idx][data_num:data_num+val_data_num]
    else:
        train_data = np.concatenate((train_data, mixed_data[0][idx][:data_num]), axis=0)
        train_label = np.concatenate((train_label, mixed_data[1][idx][:data_num]), axis=0) 
        val_data = np.concatenate((val_data, mixed_data[0][idx][data_num:data_num+val_data_num]), axis=0)
        val_label = np.concatenate((val_label, mixed_data[1][idx][data_num:data_num+val_data_num]), axis=0)   
    
train_data, train_label = shuffle(train_data, train_label)
print(train_data.shape, train_label.shape, num_class)

Number of slices : 10
(4000, 28, 28, 1) (4000, 10) 10


# Define slices

In [4]:
slice_desc = []
a = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

for i in range(num_class):
    slice_desc.append('Slice: %s' % (a[i]))
    print('Slice: %s, Initial size: %s' % (a[i], initial_data_array[i]))

Slice: T-shirt/top, Initial size: 400
Slice: Trouser, Initial size: 400
Slice: Pullover, Initial size: 400
Slice: Dress, Initial size: 400
Slice: Coat, Initial size: 400
Slice: Sandal, Initial size: 400
Slice: Shirt, Initial size: 400
Slice: Sneaker, Initial size: 400
Slice: Bag, Initial size: 400
Slice: Ankle boot, Initial size: 400


# Original ( with no data acquisition ) 

In [5]:
lr = 0.03
budget = 0
cost_func = [1] * num_class

uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=20, batch_size=32, lr=lr, epochs=2000)

Method: Uniform, Budget: 0
======= Collect Data =======
[0 0 0 0 0 0 0 0 0 0]
======= Performance =======
Loss: 0.56687 (0.01070), Average EER: 0.24029 (0.00746), Max EER: 0.48054 (0.03469)



# System T Demo on Fashion-MNIST

## Use 3000 budget, lambda=0.001, "Moderate" strategy

In [6]:
cost_func = [1] * num_class
lr = 0.03
budget = 3000
method = 'Moderate'

st = System_T((train_data, train_label), (val_data, val_label), val_data_dict, initial_data_array, num_class, add_data_dict)
st.selective_collect(budget=budget, k=10, batch_size=32, lr = lr, epochs=0.5, cost_func=cost_func, 
                 Lambda=0.001, num_iter=20, slice_desc=slice_desc, strategy=method, show_figure=False)

======= Collect Data =======
[ 45  79 334 256 448 170 490  91  99  66]
Total Cost: 2078, Remaining Budget: 922
======= Collect Data =======
[435   0 231  50   0   0   0   7 111  88]
Total Cost: 922, Remaining Budget: 0

======= Performance =======
[480.  79. 565. 306. 448. 170. 490.  98. 210. 154.]
Number of iteration: 2
Strategy: Moderate, C: 0.001, Budget: 3000
Loss: 0.48003 (0.00712), Average EER: 0.18996 (0.00490), Max EER: 0.42244 (0.02530)



# Baseline: Uniform ( = Water filling )

## For a basic setting, Uniform method is equivalent to Water filling method

In [7]:
lr = 0.03
budget = 3000
cost_func = [1] * num_class

uni = Baseline((train_data, train_label), (val_data, val_label), val_data_dict, 
                initial_data_array, num_class, add_data_dict, method='Uniform')
uni.performance(budget=budget, cost_func=cost_func, num_iter=20, batch_size=32, lr=lr, epochs=0.5)

Method: Uniform, Budget: 3000
======= Collect Data =======
[300 300 300 300 300 300 300 300 300 300]
======= Performance =======
Loss: 0.48434 (0.00825), Average EER: 0.22760 (0.00886), Max EER: 0.45218 (0.03128)



# Summary of results

| Method | Loss | Avg.EER |
|:---------------------:|:---------------------:|:---------------------:|
| Original | 0.56687 (± 0.00535) | 0.24029 (± 0.00373) |
| Uniform | 0.48434 (± 0.00413) | 0.22760 (± 0.00443) |
| Water filling | 0.48434 (± 0.00413) | 0.22760 (± 0.00443) |
| Moderate (ours) | 0.48003 (± 0.00356) | 0.18996 (± 0.00245) |